In [37]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from copy import deepcopy
import random
import pickle

In [14]:
initial_map = {
    "S": "Sam",
    "T": "Tejal",
    "Wa": "WillA",
    "Wc": "WillC",
    "Os": "Osmond",
    "N": "Nathan",
    "C": "Chris",
    "M": "Matt",
    "H": "Harrison",
    "J": "Jenny",
    "Ld": "Lorenzod",
    "Lf": "LorenzoF",
    "Sh": "Shruti",
    "Sy": "Shreyas",
    "X": "Xavier",
    "Oz": "Ozan"
}

name_to_id = {}
for i, initial in enumerate(initial_map):
    name_to_id[initial_map[initial]] = i

roles = ["Percival", "Morgana", "Merlin", "Generic Good", "Generic Evil"]

In [5]:
log = pd.read_csv("Avalon_Results.csv")

In [34]:
data = []
labels = []
current_game_number = 0
current_game = {}
for role in roles:
    current_game[role] = np.zeros(len(initial_map))
result = -1
current_game_number = 0
started_logging = True
for row_num in range(len(log)):
    
    row = log.iloc[row_num]
    
    if row["#"] != 7:
        continue
    
    if int(row["Game#"]) != current_game_number:
        if started_logging:
            labels.append(result)
            data.append(deepcopy(current_game))
        for role in roles:
            current_game[role] = np.zeros(len(initial_map))
        result = 0
        current_game_number = int(row["Game#"])
        started_logging = False
    
    person = row["Person"]
    merlin = initial_map[row["Merlin"]]
    percival = initial_map[row["Percival"]]
    morgana = initial_map[row["Morgana"]]
    result = 1 if row["Winner"] == "good" else 0
    
    if person == morgana:
        current_game["Morgana"][name_to_id[person]] = 1
    elif row["Side"] == "Evil":
        current_game["Generic Evil"][name_to_id[person]] = 1
    elif person == merlin:
        current_game["Merlin"][name_to_id[person]] = 1
    elif person == percival:
        current_game["Percival"][name_to_id[person]] = 1
    else:
        current_game["Generic Good"][name_to_id[person]] = 1
    started_logging = True
        

In [35]:
with open("dataset.p", "wb") as f:
    pickle.dump(data, f)

In [137]:
# THE MODEL
class AvalonModel(nn.Module):
    
    def __init__(self):
        
        # initializes nn.Module
        super().__init__() 
        num_players = len(name_to_id)
        
        self.overall_scores = nn.Parameter(torch.zeros([num_players], dtype=torch.float).requires_grad_())
        self.merlin_adjustments = nn.Parameter(torch.zeros([num_players], dtype=torch.float).requires_grad_())
        self.morgana_adjustments = nn.Parameter(torch.zeros([num_players], dtype=torch.float).requires_grad_())
        self.percival_adjustments = nn.Parameter(torch.zeros([num_players], dtype=torch.float).requires_grad_())
        self.bad_adjustments = nn.Parameter(torch.zeros([num_players], dtype=torch.float).requires_grad_())
        
        # final layer
        self.softmax = nn.Softmax(dim=0)
        
            
    def forward(self, batch):
        
        #----------------------CREATE INPUT VECTORS ----------------------#
        
        # create a tensor that has 1s for the bad guy ids and 0s for good guys
        bad_guys_one_hot = torch.tensor([example["Generic Evil"] + example["Morgana"] for example in batch], dtype=torch.float)
        
        # create a tensor that has 1s for the good guy ids and 0s for bad guys
        good_guys_one_hot = torch.tensor([example["Generic Good"] + example["Merlin"] + example["Percival"] for example in batch], dtype=torch.float)
        
        # create a tensor that has 1s for merlin and 0s for everyone else
        merlin_one_hot = torch.tensor([example["Merlin"] for example in batch], dtype=torch.float)

        # create a tensor that has 1s for percival and 0s for everyone else
        percival_one_hot = torch.tensor([example["Percival"] for example in batch], dtype=torch.float)

        # create a tensor that has 1s for morgana and 0s for everyone else
        morgana_one_hot = torch.tensor([example["Morgana"] for example in batch], dtype=torch.float)

        #----------------------MAKE SCORE ADJUSTMENTS----------------------#
        
        current_scores = self.overall_scores
        
        # update with the bad guy adjustments
        current_scores_1 = current_scores + (bad_guys_one_hot * self.bad_adjustments.unsqueeze(0))
        
        # update with the merlin adjustment
        current_scores_2 = current_scores_1 + (merlin_one_hot * self.merlin_adjustments.unsqueeze(0))

        # update with the percival adjustment
        current_scores_3 = current_scores_2 + (percival_one_hot * self.percival_adjustments.unsqueeze(0))

        # update with the morgana adjustment
        current_scores_4 = current_scores_3 + (morgana_one_hot * self.morgana_adjustments.unsqueeze(0))

        
        #----------------------CALCULATE PROBABILITIES----------------------#
        
        # find bad guy total scores and adjust for game size
        bad_guy_total_score = torch.sum((bad_guys_one_hot * current_scores_4), dim=1) / 3
        
        # find good guy total scores
        good_guy_total_score = torch.sum((good_guys_one_hot * current_scores_4), dim=1) / 4
        
        # find overall probability of winning -- index 0 is odds a good guy wins, index 1 is odds a bad guy wins
        final_scores = torch.stack([good_guy_total_score, bad_guy_total_score])
        probs = self.softmax(final_scores)
        
        return probs

In [417]:
baseline_reg_penalty = 0
bad_guy_reg_penalty = 0.01
merlin_reg_penalty = 0.01
percival_reg_penalty = 0.025
morgana_reg_penalty = 0.025

In [418]:
# DECLARE INSTANCE OF MODEL
model = AvalonModel()

# DECLARE OPTIMIZING VARIABLES
agent_optimizer = optim.AdamW(model.parameters(), lr=10e-5) # change the learning rate if you'd like
criterion = nn.MSELoss() # this doesn't include the regularization terms yet

In [419]:
validation_set_nums = random.sample(range(len(data)), k=32)
train_set_nums = [num for num in filter(lambda x: x not in validation_set_nums, range(len(data)))]

In [420]:
validate = False
check_in_time = 100
previous_loss = 1
loss_counts = 0
for i in range(10000):
    
    if validate:
        if i % check_in_time == 0:
            batch = [data[id_num] for id_num in validation_set_nums]
            batch_labels = torch.tensor([labels[id_num] for id_num in validation_set_nums], dtype=torch.float)
            with torch.no_grad():
                prediction = model(batch)
                loss = criterion(prediction[0, :], batch_labels)
            print(loss.item())
            loss_counts = (loss_counts + 1) if loss.item() > previous_loss else 0
            previous_loss = loss.item()
            if loss_counts >= 3:
                print("Run for {} iterations".format(i))
                break
        
    game_ids = random.sample(range(len(data)) if not validate else train_set_nums, k=64)
    batch = [data[id_num] for id_num in game_ids]
    batch_labels = torch.tensor([labels[id_num] for id_num in game_ids], dtype=torch.float)

    # feed the game into your model and get the predictions
    prediction = model(batch)

    #print(model.percival_adjustments.tensor)

    # calculate loss
    loss = criterion(prediction[0, :], batch_labels) + \
        (torch.norm(model.overall_scores) * baseline_reg_penalty) + \
        (torch.norm(model.bad_adjustments) * bad_guy_reg_penalty) + \
        (torch.norm(model.merlin_adjustments) * merlin_reg_penalty) + \
        (torch.norm(model.percival_adjustments) * percival_reg_penalty) + \
        (torch.norm(model.morgana_adjustments) * morgana_reg_penalty)

    # update
    loss.backward()
    agent_optimizer.step()
    agent_optimizer.zero_grad()

In [468]:
main_players = ["Sam", "WillA", "Tejal", "Matt", "Shruti", "Harrison", "Ozan", "LorenzoF", "Shreyas", "Xavier"]
results = {}
split_results = {}
overall_results = [0, 0]
for player in main_players:
    results[player] = [0, 0]
    split_results[player] = {}
    for role in roles:
        split_results[player][role] = [0, 0]

In [469]:
num_games = 100000
best_good_odds = 0
worst_good_odds = 1
best_good_lineup = {}
worst_good_lineup = {}
for game_num in range(num_games):
    players_this_game = random.sample(main_players, k=7)
    current_game = {}
    current_game_in_words = {}
    for role in roles:
        current_game[role] = np.zeros(len(initial_map))
    for i, player in enumerate(players_this_game):
        if i == 0:
            current_game["Merlin"][name_to_id[player]] = 1
            current_game_in_words[player] = "Merlin"
        if i == 1:
            current_game["Percival"][name_to_id[player]] = 1
            current_game_in_words[player] = "Percival"
        if i in [2, 3]:
            current_game["Generic Good"][name_to_id[player]] = 1
            current_game_in_words[player] = "Generic Good"
        if i in [4, 5]:
            current_game["Generic Evil"][name_to_id[player]] = 1
            current_game_in_words[player] = "Generic Evil"
        if i == 6:
            current_game["Morgana"][name_to_id[player]] = 1
            current_game_in_words[player] = "Morgana"
    
    with torch.no_grad():
        good_odds, _ = model([current_game])

    random_num = random.random()
    for i, player in enumerate(players_this_game):
        overall_results[0 if random_num < good_odds.item() else 1] += 1
        if i < 4:
            results[player][0 if random_num < good_odds.item() else 1] += 1
            split_results[player][current_game_in_words[player]][0 if random_num < good_odds.item() else 1] += 1
        else:
            results[player][1 if random_num < good_odds.item() else 0] += 1
            split_results[player][current_game_in_words[player]][1 if random_num < good_odds.item() else 0] += 1
    
    if good_odds.item() > best_good_odds:
        best_good_odds = good_odds.item()
        best_good_lineup = current_game_in_words
    if good_odds.item() < worst_good_odds:
        worst_good_odds = good_odds.item()
        worst_good_lineup = current_game_in_words

In [470]:
for player in results:
    print("{}: {}-{}".format(player, results[player][0], results[player][1]))

Sam: 37670-32172
WillA: 32691-37288
Tejal: 37830-32220
Matt: 37441-32741
Shruti: 36717-33338
Harrison: 31728-38304
Ozan: 38160-31825
LorenzoF: 32807-37275
Shreyas: 32721-37186
Xavier: 32276-37610


In [471]:
print("Overall")
sorted_players = sorted([player for player in results], key=lambda player: results[player][0] * 100 / (results[player][0] + results[player][1]), reverse=True)
for i, player in enumerate(sorted_players):
    print("{}. {}: {}%".format(i + 1, player, round(results[player][0] * 100 / (results[player][0] + results[player][1]), 1)))


Overall
1. Ozan: 54.5%
2. Tejal: 54.0%
3. Sam: 53.9%
4. Matt: 53.3%
5. Shruti: 52.4%
6. LorenzoF: 46.8%
7. Shreyas: 46.8%
8. WillA: 46.7%
9. Xavier: 46.2%
10. Harrison: 45.3%


In [479]:
role = "Merlin"
print(role)
sorted_players = sorted([player for player in results], key=lambda player: split_results[player][role][0] * 100 / (split_results[player][role][0] + split_results[player][role][1]), reverse=True)
for i, player in enumerate(sorted_players):
    print("{}. {}: {}%".format(i + 1, player, round(split_results[player][role][0] * 100 / (split_results[player][role][0] + split_results[player][role][1]), 1)))
    

Merlin
1. Ozan: 54.2%
2. Tejal: 53.7%
3. Sam: 53.5%
4. Matt: 53.1%
5. Shruti: 51.4%
6. Shreyas: 47.6%
7. LorenzoF: 47.3%
8. WillA: 47.0%
9. Xavier: 46.6%
10. Harrison: 45.9%


In [480]:
role = "Percival"
print(role)
sorted_players = sorted([player for player in results], key=lambda player: split_results[player][role][0] * 100 / (split_results[player][role][0] + split_results[player][role][1]), reverse=True)
for i, player in enumerate(sorted_players):
    print("{}. {}: {}%".format(i + 1, player, round(split_results[player][role][0] * 100 / (split_results[player][role][0] + split_results[player][role][1]), 1)))
    

Percival
1. Tejal: 53.9%
2. Sam: 53.5%
3. Ozan: 53.5%
4. Matt: 52.3%
5. Shruti: 51.6%
6. LorenzoF: 48.0%
7. WillA: 48.0%
8. Shreyas: 47.1%
9. Xavier: 46.3%
10. Harrison: 46.0%


In [484]:
role = "Morgana"
print(role)
sorted_players = sorted([player for player in results], key=lambda player: split_results[player][role][0] * 100 / (split_results[player][role][0] + split_results[player][role][1]), reverse=True)
for i, player in enumerate(sorted_players):
    print("{}. {}: {}%".format(i + 1, player, round(split_results[player][role][0] * 100 / (split_results[player][role][0] + split_results[player][role][1]), 1)))
    

Morgana
1. Ozan: 55.5%
2. Sam: 54.3%
3. Tejal: 54.2%
4. Matt: 54.2%
5. Shruti: 52.3%
6. LorenzoF: 46.5%
7. Shreyas: 46.2%
8. Xavier: 45.8%
9. WillA: 45.6%
10. Harrison: 44.9%


In [482]:
role = "Generic Good"
print(role)
sorted_players = sorted([player for player in results], key=lambda player: split_results[player][role][0] * 100 / (split_results[player][role][0] + split_results[player][role][1]), reverse=True)
for i, player in enumerate(sorted_players):
    print("{}. {}: {}%".format(i + 1, player, round(split_results[player][role][0] * 100 / (split_results[player][role][0] + split_results[player][role][1]), 1)))
    

Generic Good
1. Ozan: 54.2%
2. Sam: 53.8%
3. Tejal: 53.4%
4. Matt: 53.0%
5. Shruti: 52.6%
6. WillA: 47.0%
7. Shreyas: 47.0%
8. LorenzoF: 46.9%
9. Xavier: 46.8%
10. Harrison: 45.7%


In [483]:
role = "Generic Evil"
print(role)
sorted_players = sorted([player for player in results], key=lambda player: split_results[player][role][0] * 100 / (split_results[player][role][0] + split_results[player][role][1]), reverse=True)
for i, player in enumerate(sorted_players):
    print("{}. {}: {}%".format(i + 1, player, round(split_results[player][role][0] * 100 / (split_results[player][role][0] + split_results[player][role][1]), 1)))
    

Generic Evil
1. Ozan: 55.0%
2. Tejal: 54.7%
3. Sam: 54.3%
4. Matt: 53.9%
5. Shruti: 53.1%
6. Shreyas: 46.4%
7. WillA: 46.2%
8. LorenzoF: 46.0%
9. Xavier: 45.4%
10. Harrison: 44.4%


In [477]:
print("Overall")
print("Good wins {}% of the time".format(round(overall_results[0] * 100 / (overall_results[0] + overall_results[1]),2)))


Overall
Good wins 50.04% of the time


In [478]:
print("Best lineup for good")
print("Odds of good winning are {}%".format(round(best_good_odds * 100)))
for role in best_good_lineup:
    print("{}: {}".format(role, best_good_lineup[role]))
print()
print("Best lineup for evil")
print("Odds of evil winning are {}%".format(round((1 - worst_good_odds) * 100)))
for role in worst_good_lineup:
    print("{}: {}".format(role, worst_good_lineup[role]))

Best lineup for good
Odds of good winning are 74%
Tejal: Merlin
Sam: Percival
Ozan: Generic Good
Matt: Generic Good
Shreyas: Generic Evil
Xavier: Generic Evil
Harrison: Morgana

Best lineup for evil
Odds of evil winning are 74%
WillA: Merlin
Harrison: Percival
Shreyas: Generic Good
Xavier: Generic Good
Ozan: Generic Evil
Tejal: Generic Evil
Sam: Morgana


In [485]:
model.merlin_adjustments

Parameter containing:
tensor([-1.3990e-05,  7.7936e-06, -8.0841e-06,  0.0000e+00, -1.4650e-05,
         3.5414e-06,  0.0000e+00, -1.9425e-05, -2.9500e-06,  9.0133e-07,
         0.0000e+00, -1.3078e-05,  1.2378e-05, -2.0031e-05,  1.8319e-05,
        -3.3360e-06], requires_grad=True)

In [486]:
model.overall_scores

Parameter containing:
tensor([ 0.5246,  0.5680, -0.4066, -0.6460, -0.8593, -0.3879,  0.0000,  0.4932,
        -0.5770, -0.1518, -0.6510, -0.3529,  0.2990, -0.4113, -0.4377,  0.6105],
       requires_grad=True)